In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [36]:
from pymongo import MongoClient
from tqdm import tqdm

In [6]:
db = MongoClient()

In [7]:
row = next(db.datasets.elementy.find().limit(1))

In [8]:
row.keys()

dict_keys(['_id', 'markdown', 'modalities', 'url', 'summary', 'title'])

In [ ]:
db.datasets.habrahabr.drop_index("markdown_text")

In [ ]:
db.datasets.postnauka.drop_index("markdown_text")

In [21]:
%%time

for collection_name in ("postnauka", "habrahabr", "elementy"):
    db.datasets[collection_name].create_index([("markdown", "text")], default_language="russian")

CPU times: user 1.73 ms, sys: 1.24 ms, total: 2.97 ms
Wall time: 2.74 ms


In [22]:
for row in db.datasets.postnauka.find({"authors_names": {"$elemMatch": {"$eq": "Константин Воронцов"}}}):
    print(row["title"])

Математические методы прогнозирования объемов продаж


In [23]:
%%time

for row in db.datasets.postnauka.find({'$text': {'$search': 'анализы данных'}},
                           { "score": { "$meta": "textScore" }}).sort([('score', {'$meta': 'textScore'})]).limit(5):
    print(row["_id"], row["title"], row["score"])

pn_1709 Анализ данных как область знания 2.062361139347119
pn_2537 Анализ данных 2.0473438112134357
pn_1887 FAQ: Анализ данных 2.0436088319431223
pn_681 Главы | Контроль 2.030854885057471
pn_658 Анализ социальных сетей в интернете 2.0223983893293065
CPU times: user 4.43 ms, sys: 1.52 ms, total: 5.94 ms
Wall time: 89.8 ms


In [71]:
%%time

if not ("markdown_text" in db.model.all_docs.index_information().keys()):
    db.model.all_docs.create_index([("markdown", "text")], default_language='russian')
    print("done")

done
CPU times: user 663 ms, sys: 2.3 s, total: 2.96 s
Wall time: 14.3 s


In [25]:
from server.artm_lib import ArtmDataSource, ArtmBridge, ArtmModel

In [ ]:
ads = ArtmDataSource()

In [ ]:
model = ArtmModel("../server/hartm")

In [101]:
%%time

res = ads.search_query_in_models_docs(query = "биология", limit=100)
model.get_topics_by_docs_ids(res)

CPU times: user 156 ms, sys: 66.1 ms, total: 223 ms
Wall time: 221 ms


In [12]:
theta, thresholds, topics, tid_lid = model.get_topics_by_doc_id("azaza")

/Users/aksholokhov/Local Docs/repos/rysearch/experiments/server/artm_lib.py:79: DeprecationWarning: invalid escape sequence \d
  if regex.match("^topic_\d+$", tid):


In [16]:
topics_for_docs = []

for doc in res:
    if doc["doc_id"] not in thresholds.index:
        continue
    topics_for_doc = {}
    comparsion = theta[doc["doc_id"]] > thresholds[doc["doc_id"]]
    topics_for_doc["doc_id"] = doc["doc_id"]
    last_level_topics = list(comparsion[comparsion == True].index)
    levels_count = tid_lid[last_level_topics[0]][0] + 1
    topics_for_doc["level_%d"%(levels_count-1)] = last_level_topics  
    for lid in range(1, levels_count)[::-1]:
        current_level = topics_for_doc["level_%d"%(lid)]
        higher_level = []
        for topic in current_level:
            higher_level += topics[topic]["parents"]
        topics_for_doc["level_%d"%(lid-1)] = higher_level
    topics_for_docs.append(topics_for_doc)

In [152]:
res = bridge.data_source.search_query_in_models_docs("машинное обучение")

theta = bridge.model.theta
doc_theta = bridge.model._doc_theta
thresholds = bridge.model._doc_thresholds
topics = bridge.model.topics
tid_lid = bridge.model._to_lid_tid_map


import pandas as pd 
import numpy as np

lowest_level_counter = pd.Series(np.zeros(len(doc_theta.index)), index = doc_theta.index)

for doc in res:
    if doc["doc_id"] not in thresholds.index:
        continue
    topics_for_doc = {}
    lowest_level_counter += (doc_theta[doc["doc_id"]] > thresholds[doc["doc_id"]]).map(lambda x: 1 if x else 0)

levels_count = tid_lid[lowest_level_counter.index[0]][0] + 1
    
answer = pd.Series(np.zeros(len(theta.index)), index = theta.index)
answer[lowest_level_counter.index] = lowest_level_counter

for lid in range(0, levels_count-1)[::-1]:
    curr_level_topics = list(filter(lambda x: x.startswith("level_%d_t"%(lid)), answer.index))
    for topic in curr_level_topics:
        for child in topics[topic]["children"]:
            answer[topic] += answer[child]

for lid in range(0, levels_count)[::-1]:
    curr_level_topics = list(filter(lambda x: x.startswith("level_%d_t"%(lid)), answer.index))
    answer[curr_level_topics] /= sum(answer[curr_level_topics])

dict(answer)

In [18]:
for doc in topics_for_docs:
    level_0_topics = []
    for topic in doc["level_1_topics"]:
        level_0_topics += topics[topic]["parents"]
    doc["level_0_topics"] = level_0_topics

In [24]:
topics["level_0_topic_3"]

{'children': ['level_1_topic_3'],
 'level_id': 0,
 'parents': [],
 'top_words': ['культура', 'литература', 'культурология'],
 'weight': 361}

## Мержинг коллекций 

In [26]:
%%time

bridge = ArtmBridge("../server/hartm")

CPU times: user 7.56 s, sys: 633 ms, total: 8.19 s
Wall time: 8.75 s


In [38]:
%%time

all_docs_ids = set(bridge._model._doc_theta.columns)

for collection in db.datasets.list_collection_names():
    for doc in tqdm(db.datasets[collection].find()):
        if doc["_id"] in all_docs_ids:
            db.model.all_docs.insert_one(doc)

2017it [00:02, 951.00it/s]
94785it [00:28, 3312.63it/s]
2976it [00:04, 643.45it/s]

CPU times: user 22 s, sys: 3.12 s, total: 25.1 s
Wall time: 35.4 s


In [40]:
%%time

db.model.all_docs.create_index([("markdown", "text")], default_language="russian")

CPU times: user 367 ms, sys: 460 ms, total: 827 ms
Wall time: 15.9 s


'markdown_text'

In [45]:
db.model.all_docs.count_documents({})

5188

---